In [1]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*
import jline.solvers.mva.*
import jline.util.matrix.*

In [2]:
// Fork-Join network example 10
val model = Network("model")

// Create network nodes
val delay = Delay(model, "Delay1")
val queue1 = Queue(model, "Queue1", SchedStrategy.FCFS)
val queue2 = Queue(model, "Queue2", SchedStrategy.FCFS)
val queue3 = Queue(model, "Queue3", SchedStrategy.FCFS)
val fork = Fork(model, "Fork")
val join = Join(model, "Join", fork)

val jobclass1 = ClosedClass(model, "class1", 10, delay, 0)

In [3]:
// Service configurations
queue1.setService(jobclass1, Exp(1.0))
queue2.setService(jobclass1, Exp(2.0))
queue3.setService(jobclass1, Exp(1.0))
delay.setService(jobclass1, Exp(0.5))

In [4]:
// Set routing matrix with serial queue2->queue3 routing
val P = model.initRoutingMatrix()

P.set(jobclass1, jobclass1, delay, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, queue2, queue3, 1.0)  // Serial routing: queue2 -> queue3
P.set(jobclass1, jobclass1, queue3, join, 1.0)
P.set(jobclass1, jobclass1, join, delay, 1.0)

model.link(P)

In [ ]:
// Solve with multiple methods
val solvers = arrayOf(
    JMT(model, "seed", 23000),
    MVA(model)
)

for ((i, solver) in solvers.withIndex()) {
    println("SOLVER ${i+1}: ${solver.name}")
    try {
        val avgTable = solver.avgTable
        avgTable.print()
    } catch (e: Exception) {
        println("Error: ${e.message}")
    }
}